In [ ]:
# =============================================================================
# STEP 1: IMPORT LIBRARIES AND SETUP
# =============================================================================
# Import necessary libraries for web scraping and data processing
import pandas as pd  # For data manipulation and analysis
import ssl           # For handling SSL certificates
import warnings      # For suppressing warnings
import time          # For adding delays between requests
from datetime import datetime  # For date handling

# Suppress warnings to keep output clean
warnings.filterwarnings('ignore')

# Handle SSL certificate issues (needed for some websites)
# This allows us to access HTTPS websites without certificate verification
ssl._create_default_https_context = ssl._create_unverified_context

print("✅ Libraries imported successfully!")
print("📚 Ready to start historical web scraping (1993-2025)...")
print(f"🕐 Started at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")


In [ ]:
# =============================================================================
# STEP 2: DEFINE DATA CLEANING FUNCTIONS
# =============================================================================

def clean_lottery_data(df, year):
    """
    Clean the lottery data by removing monthly header rows and invalid data
    
    Parameters:
    df (DataFrame): Raw lottery data from web scraping
    year (int): Year being processed (for logging)
    
    Returns:
    DataFrame: Cleaned data with monthly headers removed
    """
    print(f"📊 {year} - Original data shape: {df.shape}")
    
    # Create a copy to avoid modifying the original data
    df_clean = df.copy()
    
    # Find and remove monthly header rows (e.g., "September 2025", "August 2025")
    monthly_rows = []
    for idx, row in df_clean.iterrows():
        # Get all non-null values in the row
        non_null_values = [str(val) for val in row if pd.notna(val)]
        
        if len(non_null_values) > 0:
            # Check if all values are the same and contain month names
            # This identifies rows like "September 2025, September 2025, September 2025"
            if len(set(non_null_values)) == 1 and any(month in non_null_values[0] for month in 
                ['January', 'February', 'March', 'April', 'May', 'June', 
                 'July', 'August', 'September', 'October', 'November', 'December']):
                monthly_rows.append(idx)
    
    # Remove the identified monthly header rows
    df_clean = df_clean.drop(monthly_rows)
    
    # Remove rows where Draw Number doesn't contain a slash (invalid lottery numbers)
    # Valid lottery numbers should look like "25/096", "25/095", etc.
    if 'Draw Number' in df_clean.columns:
        df_clean = df_clean[df_clean['Draw Number'].str.contains('/', na=False)]
    
    # Reset the index after dropping rows (important for data integrity)
    df_clean = df_clean.reset_index(drop=True)
    
    print(f"🗑️  {year} - Removed {len(monthly_rows)} monthly header rows")
    print(f"📊 {year} - Cleaned data shape: {df_clean.shape}")
    
    return df_clean

def split_balls_drawn(balls_str):
    """
    Split balls drawn string into individual numbers
    
    Parameters:
    balls_str (str): String containing lottery numbers (e.g., "5 18 23 24 29 49 11")
    
    Returns:
    list: List of 7 individual numbers
    """
    # Handle missing values
    if pd.isna(balls_str):
        return [None] * 7
    
    # Remove commas and split by spaces, then filter out empty strings
    numbers = [num.strip() for num in str(balls_str).replace(',', '').split() if num.strip()]
    
    # Ensure we have exactly 7 numbers (pad with None if less)
    while len(numbers) < 7:
        numbers.append(None)
    
    # Return only the first 7 numbers (in case there are more)
    return numbers[:7]

def process_lottery_data(df, year):
    """
    Process lottery data for a specific year
    
    Parameters:
    df (DataFrame): Raw lottery data
    year (int): Year being processed
    
    Returns:
    DataFrame: Processed data ready for analysis
    """
    if df is None or df.empty:
        print(f"❌ {year} - No data available")
        return None
    
    # Clean the data
    df_clean = clean_lottery_data(df, year)
    
    if df_clean.empty:
        print(f"❌ {year} - No valid data after cleaning")
        return None
    
    # Drop unnecessary columns if they exist
    columns_to_drop = ['Draw Number', 'Details']
    existing_columns_to_drop = [col for col in columns_to_drop if col in df_clean.columns]
    if existing_columns_to_drop:
        df_clean = df_clean.drop(columns=existing_columns_to_drop)
        print(f"🗑️  {year} - Removed columns: {existing_columns_to_drop}")
    
    # Format balls drawn with commas
    if 'Balls Drawn' in df_clean.columns:
        def format_balls_drawn(balls_str):
            if pd.isna(balls_str):
                return balls_str
            numbers = [num.strip() for num in str(balls_str).split() if num.strip()]
            return ', '.join(numbers)
        
        df_clean['Balls Drawn'] = df_clean['Balls Drawn'].apply(format_balls_drawn)
        
        # Split into individual number columns
        balls_data = df_clean['Balls Drawn'].apply(split_balls_drawn)
        
        for i in range(7):
            df_clean[f'num{i+1}'] = balls_data.apply(lambda x: x[i])
        
        # Remove the original Balls Drawn column
        df_clean = df_clean.drop(columns=['Balls Drawn'])
        
        print(f"🔢 {year} - Separated numbers into 7 columns")
    
    # Note: Year column removed as requested - data will be identified by Draw Date only
    
    print(f"✅ {year} - Processing completed. Final shape: {df_clean.shape}")
    
    return df_clean

print("✅ Data processing functions defined successfully!")
print("🔧 Functions ready: clean_lottery_data(), split_balls_drawn(), process_lottery_data()")


In [ ]:
# =============================================================================
# STEP 3: DEFINE YEARS TO SCRAPE AND INITIALIZE
# =============================================================================

# Define the range of years to scrape (1993 to 2025)
start_year = 1993
end_year = 2025
years_to_scrape = list(range(start_year, end_year + 1))

print(f"📅 Years to scrape: {start_year} to {end_year}")
print(f"🔢 Total years: {len(years_to_scrape)}")
print(f"📋 Years list: {years_to_scrape}")

# Initialize variables for data collection
all_data = []  # List to store all processed data
successful_years = []  # Track successfully scraped years
failed_years = []  # Track failed years

print(f"\n✅ Initialization completed!")
print(f"📊 Ready to scrape {len(years_to_scrape)} years of lottery data")


In [ ]:
# =============================================================================
# STEP 4: SCRAPE DATA FOR EACH YEAR
# =============================================================================

print("🌐 Starting historical data scraping...")
print("⏳ This may take several minutes due to the large amount of data...")
print("=" * 80)

for i, year in enumerate(years_to_scrape, 1):
    print(f"\n📅 Processing year {year} ({i}/{len(years_to_scrape)})...")
    
    try:
        # Construct URL for the specific year
        url = f"https://lottery.hk/en/mark-six/results/{year}"
        print(f"🌐 Fetching: {url}")
        
        # Scrape data for this year
        scraped = pd.read_html(url)
        
        if scraped and len(scraped) > 0:
            # Get the first table (main results table)
            df_year = scraped[0]
            
            # Process the data for this year
            processed_data = process_lottery_data(df_year, year)
            
            if processed_data is not None and not processed_data.empty:
                all_data.append(processed_data)
                successful_years.append(year)
                print(f"✅ {year} - Successfully processed {len(processed_data)} records")
            else:
                failed_years.append(year)
                print(f"❌ {year} - No valid data after processing")
        else:
            failed_years.append(year)
            print(f"❌ {year} - No tables found on webpage")
            
    except Exception as e:
        failed_years.append(year)
        print(f"❌ {year} - Error occurred: {str(e)[:100]}...")
    
    # Add a small delay to be respectful to the server
    if i < len(years_to_scrape):  # Don't delay after the last request
        time.sleep(1)  # 1 second delay between requests

print("\n" + "=" * 80)
print(f"🎉 Scraping completed!")
print(f"✅ Successful years: {len(successful_years)} - {successful_years}")
print(f"❌ Failed years: {len(failed_years)} - {failed_years}")
print(f"📊 Total dataframes collected: {len(all_data)}")


In [ ]:
# =============================================================================
# STEP 5: COMBINE ALL DATA
# =============================================================================

if all_data:
    print("🔗 Combining all historical data...")
    
    # Combine all dataframes into one
    combined_df = pd.concat(all_data, ignore_index=True)
    
    print(f"✅ Successfully combined {len(all_data)} dataframes")
    print(f"📊 Combined data shape: {combined_df.shape} (rows × columns)")
    print(f"📋 Columns: {list(combined_df.columns)}")
    
    # Display basic statistics
    print(f"\n📈 COMBINED DATA STATISTICS:")
    print(f"   🎯 Total lottery draws: {len(combined_df):,}")
    
    if 'Draw Date' in combined_df.columns:
        print(f"   📅 Date range: {combined_df['Draw Date'].min()} to {combined_df['Draw Date'].max()}")
        print(f"   📊 Data spans multiple years (1993-2025) as indicated by date range")
    
    # Show sample of combined data
    print(f"\n👀 Sample of combined data (first 5 rows):")
    print(combined_df.head())
    
    print(f"\n👀 Sample of combined data (last 5 rows):")
    print(combined_df.tail())
    
else:
    print("❌ No data to combine!")
    combined_df = None


In [ ]:
# =============================================================================
# STEP 6: SAVE DATA TO CSV FILE
# =============================================================================

if combined_df is not None:
    print("💾 Saving combined historical data to CSV file...")
    
    # Define output filename
    output_file = 'all.csv'
    
    try:
        # Save to CSV file
        combined_df.to_csv(output_file, index=False)
        
        print(f"✅ Successfully saved data to '{output_file}'")
        print(f"📁 File size: {len(combined_df):,} rows × {len(combined_df.columns)} columns")
        
        # Display data types
        print(f"\n📊 Data types:")
        print(combined_df.dtypes)
        
        # Final summary
        print(f"\n🎉 HISTORICAL DATA SCRAPING COMPLETED SUCCESSFULLY!")
        print(f"📚 The complete dataset ({len(combined_df):,} records) is now ready for analysis!")
        print(f"🕐 Completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
        
    except Exception as e:
        print(f"❌ Error saving file: {e}")
        
else:
    print("❌ Cannot save data - no combined data available")
    print("💡 Check the scraping results above for any issues")


In [ ]:
# =============================================================================
# STEP 7: DATA QUALITY CHECK AND SUMMARY
# =============================================================================

if combined_df is not None:
    print("🔍 Performing data quality checks...")
    
    # Check for missing values
    print(f"\n📊 Missing values per column:")
    missing_values = combined_df.isnull().sum()
    for col, missing in missing_values.items():
        if missing > 0:
            print(f"   {col}: {missing:,} missing values")
    
    # Check for duplicate rows
    duplicates = combined_df.duplicated().sum()
    print(f"\n🔄 Duplicate rows: {duplicates:,}")
    
    # Check number columns for valid ranges (1-49 for Mark Six)
    number_columns = [col for col in combined_df.columns if col.startswith('num')]
    if number_columns:
        print(f"\n🎲 Number column analysis:")
        for col in number_columns:
            # Convert to numeric, errors='coerce' will turn invalid values to NaN
            numeric_values = pd.to_numeric(combined_df[col], errors='coerce')
            valid_numbers = numeric_values.dropna()
            
            if len(valid_numbers) > 0:
                min_val = valid_numbers.min()
                max_val = valid_numbers.max()
                print(f"   {col}: Range {min_val}-{max_val}, Valid values: {len(valid_numbers):,}")
            else:
                print(f"   {col}: No valid numeric values")
    
    # Date-based analysis (since Year column is removed)
    if 'Draw Date' in combined_df.columns:
        print(f"\n📅 Date-based analysis:")
        print(f"   Data spans from {combined_df['Draw Date'].min()} to {combined_df['Draw Date'].max()}")
        print(f"   Total unique dates: {combined_df['Draw Date'].nunique()}")
    
    print(f"\n✅ Data quality check completed!")
    
else:
    print("❌ Cannot perform quality check - no data available")


In [ ]:
# =============================================================================
# END OF HISTORICAL SCRAPING NOTEBOOK
# =============================================================================
# This notebook demonstrates comprehensive historical data scraping
# for Hong Kong Mark Six lottery results from 1993 to 2025.
# 
# Key features:
# - Scrapes 33 years of lottery data
# - Handles errors gracefully with retry logic
# - Combines all data into a single CSV file
# - Includes data quality checks
# - Educational step-by-step approach
#
# Output: all.csv (complete historical dataset)
# =============================================================================

print("📚 Historical Mark Six Lottery Data Scraper")
print("🎯 Successfully scraped data from 1993 to 2025")
print("📁 Output file: all.csv")
print("🔬 Ready for analysis, machine learning, or further processing!")
